<a href="https://colab.research.google.com/github/seungmin001/prac_deeplearningzerotoall/blob/main/Lab5_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# code by devKya
import tensorflow as tf
import numpy as np

x_train = np.array([
    [1, 2],
    [2, 3],
    [3, 1],
    [4, 3],
    [5, 3],
    [6, 2]], dtype=np.float32)
y_train = np.array([
    [0],
    [0],
    [0],
    [1],
    [1],
    [1]], dtype=np.float32)

x_test = np.array([[5, 2]], dtype=np.float32)
y_test = np.array([[1]], dtype=np.float32)

# tf.data.Dataset 파이프라인을 이용하여 값을 입력
# from_tensor_slices 클래스 매서드를 사용하면 리스트, 넘파이, 텐서플로 자료형에서 데이터셋을 만들 수 있음
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train)) # x_train y_train 합쳐서 x_train열이 끝나면 y_train열이 옴. # batch : 한번에 학습하는 data의 개수
W = tf.Variable(tf.zeros([2, 1]), name='weight') # (x_train열개수, 결과 개수)
b = tf.Variable(tf.zeros([1]), name='bias')

# 원소의 자료구조 반환
dataset.element_spec

def logistic_regression(features):
    hypothesis = tf.sigmoid(tf.matmul(features, W) + b)
    return hypothesis


def loss_fn(features, labels):
    hypothesis = logistic_regression(features)
    cost = -tf.reduce_mean(labels * tf.math.log(hypothesis) + (1 - labels) * tf.math.log(1 - hypothesis))
    return cost

def grad(hypothesis, features, labels):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(features, labels)
    return tape.gradient(loss_value, [W,b])

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

EPOCHS = 3000

for step in range(EPOCHS + 1):
    for features, labels in iter(dataset):
        hypothesis = logistic_regression(features)
        grads = grad(hypothesis, features, labels)
        optimizer.apply_gradients(grads_and_vars=zip(grads, [W,b]))
        if step % 300 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, loss_fn(features, labels)))
            # print(hypothesis)
            
def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.int32))
    return accuracy

test_acc = accuracy_fn(logistic_regression(x_test), y_test)
print('Accuracy: {}%'.format(test_acc * 100))

Iter: 0, Loss: 0.6874
Iter: 300, Loss: 0.5054
Iter: 600, Loss: 0.4535
Iter: 900, Loss: 0.4228
Iter: 1200, Loss: 0.3992
Iter: 1500, Loss: 0.3790
Iter: 1800, Loss: 0.3608
Iter: 2100, Loss: 0.3442
Iter: 2400, Loss: 0.3288
Iter: 2700, Loss: 0.3146
Iter: 3000, Loss: 0.3013
Accuracy: 100%


In [ ]:
# my code
import tensorflow as tf
import numpy as np

# Data
x_train = np.array([
    [1, 2],
    [2, 3],
    [3, 1],
    [4, 3],
    [5, 3],
    [6, 2]], dtype=np.float32)
y_train = np.array([
    [0],
    [0],
    [0],
    [1],
    [1],
    [1]], dtype=np.float32)

x_test = np.array([[5, 2]], dtype=np.float32)
y_test = np.array([[1]], dtype=np.float32)

# Dataset
dataset= tf.data.Dataset.from_tensor_slices((x_train, y_train))
print(dataset)
dataset= dataset.batch(len(x_train))
print(dataset)


# Weight, bias
W= tf.Variable(tf.random.normal([2,1], dtype=np.float32, name='Weight'))
b= tf.Variable(tf.random.normal([1], dtype=np.float32, name='bias'))

# logistic regression
def logistic_regression(features):
    # sigmoid = 1/(1+e^(-z)) , z = X*W + b
    hypothesis= tf.sigmoid( tf.matmul(features,W) + b ) 
    return hypothesis

# cost function
def cost_func(features, labels):
    hypothesis = logistic_regression(features)
    # cost = y * - log(h(x)) + (1-y) * - log(1 - h(x)) # y=1일 때 cost 0되는 함수+ y=0일 때 cost 0되는 함수 온전하게 반영
    # cost function은 cost의 reduce_mean
    cost = -tf.reduce_mean(labels * tf.math.log(hypothesis) + (1 - labels) * tf.math.log(1 - hypothesis))
    return cost

def grad(features, labels):
    with tf.GradientTape() as tape:
        cost = cost_func(features, labels)
    return tape.gradient(cost, [W,b])

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

EPOCHS= 3000
# 반복 learning
for step in range(EPOCHS + 1):
    for features, labels in dataset: # features shape=(6,2) labels shape=(6,1)
        grads= grad(features, labels)
        optimizer.apply_gradients(grads_and_vars = zip(grads, [W,b]))
        
        if step % 300 == 0:
            print("{:5} | {:12.4f}".format(step, cost_func(features,labels)))

# 예측값, 실제값 비교해서 정확성 반환
def accuracy_fn(hypothesis, labels):
    # 0.5 기준 크면 1 작으면 0
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.int32))
    return accuracy

test_acc = accuracy_fn(logistic_regression(x_test), y_test)
print('Accuracy: {}%'.format(test_acc * 100))

<TensorSliceDataset shapes: ((2,), (1,)), types: (tf.float32, tf.float32)>
<BatchDataset shapes: ((None, 2), (None, 1)), types: (tf.float32, tf.float32)>
    0 |       6.7401
  300 |       0.5137
  600 |       0.4326
  900 |       0.3963
 1200 |       0.3725
 1500 |       0.3534
 1800 |       0.3368
 2100 |       0.3218
 2400 |       0.3079
 2700 |       0.2952
 3000 |       0.2833
Accuracy: 100%


#### 참고
batch o iter o shape=(6, 2) shape=(6, 1)  
batch x iter o shape=[2] (행이 없고 x 2개씩) matrix가 아니라 matmul연산안됨  
batch o iter x shape (6,2)  
batch x iter x shape=[2] matmul 오류  

In [ ]:
# dataset 기초

import numpy as np
import tensorflow as tf

# ndarray x 선언
x=np.array([[1.,2.],[3.,4.]])
print(x.dtype)
print(len(x))

# dataset 선언
dataset=tf.data.Dataset.from_tensor_slices(x)
dataset2=tf.data.Dataset.from_tensors(x)

print(dataset.element_spec) # 각 element 들의 속성

print(dataset) # slices는 잘라서 저장
print(dataset2) # 한꺼번에 저장

float64
2
TensorSpec(shape=(2,), dtype=tf.float64, name=None)
<TensorSliceDataset shapes: (2,), types: tf.float64>
<TensorDataset shapes: (2, 2), types: tf.float64>
